### Libraries

In [35]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
from matplotlib import gridspec
from tqdm import tqdm
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import FashionMNIST
from torchvision import datasets, transforms
import torchvision
import torch

import warnings
warnings.filterwarnings("ignore")

In [40]:
# !unzip experiment.zip -d .
%load_ext tensorboard
%tensorboard --logdir ./experiment/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 31418), started 4:53:23 ago. (Use '!kill 31418' to kill it.)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Dataset & general parameters

Скачаем dataset для восстановления Part-of-Speech (POS) тегов для токенов в предложениях.

Общий алгоритм подготовки данных прост и понятен: извлекаем слова в предложениях (токены) и их теги, строим отображение токенов и тегов в индексы (т.е. преобразуем их в последовательности чисел); далее создайм кастомный dataset и добавляем возможность padding'а через проброс кастомного `collate_fn` в `DataLoader`. 

In [6]:
# Uncomment string below to download dataset
# !wget https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz

In [7]:
from nerus import load_nerus
docs = load_nerus('nerus_lenta.conllu.gz')
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['<PAD>']

sentences = []
tags = []

cnt = 0
n_docs_max = 5000

for doc in tqdm(docs):
    cnt += 1
    for sent in doc.sents:
        sent_ = []
        tag_ = []

        for word in sent.tokens:
            tag_.append(word.pos)
            sent_.append(word.text)

        sentences.append(sent_)
        tags.append(tag_)

    if cnt > n_docs_max:
        break

5000it [00:02, 1895.01it/s]


In [8]:
set_tokens = {word for sent in sentences for word in sent}
set_tokens.difference_update(special_tokens)
list_tokens = special_tokens + list(set_tokens)

set_tags = {tag for t in tags for tag in t}
set_tags.difference_update(special_tags)
list_tags = special_tags + list(set_tags)

token_to_idx = dict(zip(list_tokens, np.arange(len(list_tokens))))
tag_to_idx = dict(zip(list_tags, np.arange(len(list_tags))))

In [9]:
train_test_boundary = int(len(sentences) * 0.9)

train_sentences = sentences[:train_test_boundary]
train_tags = tags[:train_test_boundary]
test_sentences = sentences[train_test_boundary:]
test_tags = tags[train_test_boundary:]

print(len(train_sentences), len(test_sentences))

53112 5902


In [10]:
class TaggingDataset(torch.utils.data.Dataset):
    def __init__(self, sentences, tags, token_to_idx, tag_to_idx):
        super().__init__()

        self.sentences = sentences
        self.tags = tags
        self.token_to_idx = token_to_idx
        self.tag_to_idx = tag_to_idx

        sent_index = []
        tags_index = []

        for sent in sentences:
            sequence = []

            for token in sent:
                if token in self.token_to_idx:
                    sequence.append(token_to_idx[token])
                else:
                    sequence.append(0)

            sent_index.append(sequence)

        for sent_tags in tags:
            tgs = []

            for tag in sent_tags:
                tgs.append(tag_to_idx[tag])

            tags_index.append(tgs)

        self.sent_index = sent_index
        self.tags_index = tags_index

    def __getitem__(self, idx):
        return torch.tensor(self.sent_index[idx]), torch.tensor(self.tags_index[idx])

    def __len__(self):
        return len(self.sent_index)

In [11]:
train_dataset = TaggingDataset(train_sentences, train_tags, token_to_idx, tag_to_idx)
test_dataset = TaggingDataset(test_sentences, test_tags, token_to_idx, tag_to_idx)

In [12]:
class PaddingCollator:
    def __init__(self, pad_token_id, pad_tag_id):
        self.pad_token_idx = pad_token_id
        self.pad_tag_id = pad_tag_id

    def __call__(self, batch):

        max_len = 0
        for elem in batch:
            if (len(elem[0]) > max_len):
                max_len = len(elem[0])

        new_sentences = []
        new_tags = []

        for elem in batch:
            new_sentences.append(torch.nn.functional.pad(elem[0], (0, max_len - len(elem[0])), "constant", self.pad_token_idx))
            new_tags.append(torch.nn.functional.pad(elem[1], (0, max_len - len(elem[1])), "constant", self.pad_tag_id))

        return torch.stack(new_sentences), torch.stack(new_tags)

In [13]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=False,
    drop_last=False,
    collate_fn=PaddingCollator(
        pad_token_id=token_to_idx['<PAD>'],
        pad_tag_id=tag_to_idx['<PAD>'],
    )
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    drop_last=False,
    collate_fn=PaddingCollator(
        pad_token_id=token_to_idx['<PAD>'],
        pad_tag_id=tag_to_idx['<PAD>'],
    )
)

### General training code

Код для обучения, формат которого взят с семинаров.

In [14]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    model.zero_grad()

    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    output = model(x_batch)
    output = torch.transpose(output, 1, 2)
    loss = loss_function(output, y_batch)

    loss.backward()
    optimizer.step()

    return loss.cpu().item()

In [15]:
def train_epoch(train_generator, model, loss_function, optimizer, callback):
    epoch_loss = 0
    total = 0

    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(
            model, batch_of_x.to(device), batch_of_y.to(device), optimizer, loss_function)

        if callback is not None:
            callback(model, batch_loss)

        epoch_loss += batch_loss * len(batch_of_x)
        total += len(batch_of_x)

    return epoch_loss / total

In [16]:
def trainer(count_of_epoch,
            model,
            dataset_loader,
            loss_function,
            optimizer,
            lr=0.001,
            callback=None):
    optima = optimizer(model.parameters(), lr=lr, weight_decay=1e-5)

    iterations = tqdm(range(count_of_epoch))

    for it in iterations:
        epoch_loss = train_epoch(
            train_generator=dataset_loader, model=model,
            loss_function=loss_function,
            optimizer=optima,
            callback=callback)

        iterations.set_postfix({'train epoch loss': epoch_loss})

In [24]:
def quality_of_train(dataset_loader,
                     model,
                     loss_function):
    pred = []
    real = []
    test_loss = 0
    total = 0

    for it, (sentences, tags) in enumerate(dataset_loader):
        sentences = sentences.to(device)
        tags = tags.to(device)

        output = model(sentences)

        pred.extend(torch.argmax(output, dim=2).cpu().numpy().flatten().tolist())
        real.extend(tags.cpu().numpy().flatten().tolist())

        output = torch.transpose(output, 1, 2)
        test_loss += loss_function(output, tags).cpu().item() * len(sentences)
        total += len(sentences)

    test_loss /= total

    return test_loss, pred, real

### LSTM model

In [18]:
class LSTMTagger(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size, dropout=0.0):
        super(LSTMTagger, self).__init__()

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim, batch_first=True, dropout=dropout)
        self.linear = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, x_batch):
        embeddings = self.word_embeddings(x_batch)

        d_n, (h_n, c_n) = self.lstm(embeddings)
        return self.linear(d_n)

### Tensorboard training tracking

In [38]:
class callback():
    def __init__(self, writer, dataset_loader, loss_function, delimeter=100):
        self.step = 0
        self.writer = writer
        self.delimeter = delimeter
        self.loss_function = loss_function

        self.dataset_loader = dataset_loader

    def forward(self, model, loss):
        self.step += 1
        self.writer.add_scalar('LOSS/train', loss, self.step)
        model.eval()

        if self.step % self.delimeter == 0:
            test_loss, pred, real = quality_of_train(dataset_loader=self.dataset_loader,
                                                     model=model, loss_function=self.loss_function)
            self.writer.add_scalar('LOSS/test', test_loss, self.step)

            indices = (real != tag_to_idx['<PAD>']).nonzero()

            real = np.array(real)[indices]
            pred = np.array(pred)[indices]

            self.writer.add_scalar('Valid/acc', accuracy_score(real, pred), self.step)
            self.writer.add_text('Valid/report', str(classification_report(real, pred)), self.step)

    def __call__(self, model, loss):
        return self.forward(model, loss)

### LSTM training

In [20]:
loss_function = torch.nn.CrossEntropyLoss(ignore_index=tag_to_idx['<PAD>'])
optimizer = torch.optim.Adam

In [39]:
grid = ParameterGrid({
    'embedding_dim': [150, 290, 430, 600],
    'hidden_dim': [150, 290, 430, 600],
    'dropout': [0.0, 0.18, 0.36, 0.54],
})

scores = dict()

for item in tqdm(grid):
    print(str(item))

    model = LSTMTagger(
        embedding_dim=item['embedding_dim'],
        hidden_dim=item['hidden_dim'],
        vocab_size=len(token_to_idx),
        tagset_size=len(tag_to_idx),
        dropout=item['dropout']
    )

    writer = SummaryWriter('experiment/' + str(item))

    model.float().to(device)

    call = callback(writer, test_dataloader, loss_function, delimeter=10)

    trainer(count_of_epoch=2,
        dataset_loader=train_dataloader,
        model=model,
        loss_function=loss_function,
        optimizer=optimizer,
        lr=0.001,
        callback=call)

  0%|          | 0/64 [00:00<?, ?it/s]

{'dropout': 0.0, 'embedding_dim': 150, 'hidden_dim': 150}


  2%|▏         | 1/64 [01:09<1:12:39, 69.19s/it]

{'dropout': 0.0, 'embedding_dim': 150, 'hidden_dim': 290}


  3%|▎         | 2/64 [02:28<1:17:27, 74.96s/it]

...

 97%|█████████▋| 62/64 [1:30:18<03:04, 92.34s/it]

{'dropout': 0.54, 'embedding_dim': 600, 'hidden_dim': 430}


 98%|█████████▊| 63/64 [1:32:00<01:35, 95.16s/it]

{'dropout': 0.54, 'embedding_dim': 600, 'hidden_dim': 600}


100%|██████████| 64/64 [1:33:41<00:00, 87.83s/it]


### Conclusions

Результаты получились следующие:
* Dropout не сильно влияет на точность модели, что более-менее ожидаемо.
* Чем больше размерность embedding слоя, тем больше точность (не особо сильно), что ожидаемо.
* Чем больше размерность hidden слоя, тем больше точность (не особо сильно), что ожидаемо.
* Наибольший score набрала модель с максимальной размерностью embedding слоя и hidden слоя, что подтверждает наблюдения выше.
* В целом модель хорошо справилась со своей задачей: accuracy модели превышает 0.9 на определённых наборах параметров. 
